In [2]:
!pip install kafka-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 3.6 MB/s eta 0:00:00a 0:00:01


In [3]:
!pip3 install minio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 3.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.3 MB/s eta 0:00:0000:0100:01


In [4]:
from minio import Minio

In [19]:
client = Minio("minio:9000",
    access_key="DImuDiaEs6bIJJBgCjXW",
    secret_key="HPCTwG5dobqO83MAoCz4hmpPMaJblP9RvondmMHJ",
    secure = False
)

In [20]:
client.make_bucket("bronze")

In [27]:
import json
from minio import Minio
from minio.error import S3Error
from io import BytesIO

# Nome do bucket e do objeto
bucket_name = "bronze"
object_name = "dados/clientes.json"  # Caminho e nome do arquivo no bucket

# Dados a serem gravados
dados = {
    "clientes": [
        {"nome": "João", "sobrenome": "Silva", "idade": 34, "email": "joao.silva@email.com", "data_cadastro": "2024-08-09"},
        {"nome": "Maria", "sobrenome": "Oliveira", "idade": 29, "email": "maria.oliveira@email.com", "data_cadastro": "2024-08-09"}
    ]
}

# Converter o dicionário Python em uma string JSON
json_data = json.dumps(dados)

# Converter a string JSON em bytes para poder enviar ao MinIO
json_bytes = BytesIO(json_data.encode('utf-8'))

# Fazendo upload do JSON para o MinIO
try:
    client.put_object(
        bucket_name,
        object_name,
        data=json_bytes,
        length=len(json_data),
        content_type="application/json"
    )
    print(f'JSON {object_name} enviado para o bucket {bucket_name} com sucesso!')
except S3Error as e:
    print(f'Erro ao enviar arquivo: {e}')


JSON dados/clientes.json enviado para o bucket bronze com sucesso!


In [60]:
len(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

19

In [122]:
from kafka import KafkaConsumer
TOPIC = 'teste'
BOOTSTRAP = 'kafka:9092'
ACCESS_KEY = "DImuDiaEs6bIJJBgCjXW"
SECRET_KEY = "HPCTwG5dobqO83MAoCz4hmpPMaJblP9RvondmMHJ"
from clickhouse_driver import Client
from minio import Minio
BUCKET_NAME = 'bronze-layer'

class TopicOffsetControll():
    def __init__(self, topic):
        self.topic = topic
        self.client = Client(
                host='clickhouse',       # Endereço do servidor ClickHouse
                port=9000,              # Porta padrão para conexões TCP
                user='default',         # Usuário (padrão é 'default')
                password='',            # Senha (por padrão, vazio)
                database='default',     # Nome do banco de dados (por padrão, 'default')
        )
        
    def get_offset(self):
        self.client.execute("""CREATE TABLE IF NOT EXISTS kafka_topics_offset_control (
                kafka_topic String,
                last_offset Int64,
                exec_date DEFAULT now() 
                ) ENGINE = MergeTree()
                ORDER BY exec_date """)
        offset = self.client.execute(f"SELECT max(last_offset) FROM kafka_topics_offset_control WHERE kafka_topic = '{self.topic}'")[0][0]
        return offset

    def insert_offset(self, offset):
        print(f"OFFSET {offset} ")
        self.client.execute(f"""
        INSERT INTO kafka_topics_offset_control (kafka_topic, last_offset) 
        VALUES ('{self.topic}',  {offset})
        """)
    
def insert_bronze(data):
    client = Minio("minio:9000",
        access_key=ACCESS_KEY,
        secret_key=SECRET_KEY,
        secure = False
    )
    found = client.bucket_exists(BUCKET_NAME)
    if not found:
        client.make_bucket(BUCKET_NAME)
        print("Created bucket", BUCKET_NAME)
    json_data = json.dumps(dados)
    json_bytes = BytesIO(json_data.encode('utf-8'))
    now = datetime.now()
    object_name = f'KAFKA/topic={TOPIC}/year={now.year}/month={now.month}/day={now.day}/{now.strftime("%Y-%m-%d %H:%M:%S")}.json'
    try:
        client.put_object(
            BUCKET_NAME,
            object_name,
            data=json_bytes,
            length=len(json_data),
            content_type="application/json"
        )
        print(f'JSON {object_name} enviado para o bucket {bucket_name} com sucesso!')
    except S3Error as e:
        print(f'Erro ao enviar arquivo: {e}')    
# Configuração do consumidor Kafka
try:
    consumer = KafkaConsumer(
        bootstrap_servers=[BOOTSTRAP],  # Servidor Kafka
        enable_auto_commit=True,  # Para commit automático das mensagens
        consumer_timeout_ms=10000,  #fica aguardando topicos por 10 segundos
        value_deserializer=lambda x: x.decode('utf-8')  # Deserializa as mensagens para string
    )
    # Especificar a partição e o offset inicial
    topic_controll = TopicOffsetControll(TOPIC)
    partition = 0  # Partição que deseja consumir
    offset_inicial = topic_controll.get_offset()  # Offset a partir do qual deseja começar a ler
    
    # Atribuir a partição ao consumidor
    tp = TopicPartition(TOPIC, partition)
    consumer.assign([tp])
    
    # Definir o offset inicial manualmente
    consumer.seek(tp, offset_inicial)
    
    print("Aguardando mensagens...")
    last_message = None
    # Consome mensagens do tópico
    message_list = []
    for message in consumer:
        last_message = message
        print(f"Recebido: {message.value}")
        message_list.append(message.value)
        
    insert_bronze(message.value)
    topic_controll.insert_offset(message.offset)
    
except Exception as e:
    print(e)
    raise Exception(e)
    #create_log(status = 'failure', message = e)




Aguardando mensagens...
Recebido: {"nome": "Enrico", "sobrenome": "Santos", "idade": 37, "email": "limacamila@example.net", "data_cadastro": "2024-05-20"}
Recebido: {"nome": "Isaac", "sobrenome": "Nascimento", "idade": 28, "email": "olivia70@example.org", "data_cadastro": "2024-05-17"}
Recebido: {"nome": "Brenda", "sobrenome": "Teixeira", "idade": 37, "email": "beniciofarias@example.org", "data_cadastro": "2024-06-23"}
Recebido: {"nome": "Arthur", "sobrenome": "Ramos", "idade": 65, "email": "diogomelo@example.com", "data_cadastro": "2024-06-25"}
Recebido: {"nome": "Paulo", "sobrenome": "Nunes", "idade": 52, "email": "sofiada-costa@example.net", "data_cadastro": "2024-03-14"}
Recebido: {"nome": "Sofia", "sobrenome": "Santos", "idade": 44, "email": "limathomas@example.com", "data_cadastro": "2024-03-11"}
Recebido: {"nome": "Thiago", "sobrenome": "Pires", "idade": 60, "email": "erick72@example.com", "data_cadastro": "2024-08-07"}
Recebido: {"nome": "Isadora", "sobrenome": "Ramos", "idade"

In [52]:
from datetime import datetime

In [10]:
!pip install clickhouse_driver

In [21]:



from datetime import datetime
from kafka import KafkaConsumer, TopicPartition
from clickhouse_driver import Client
from minio import Minio
from minio.error import S3Error
import json
from io import BytesIO
import os

from kafka import KafkaConsumer
TOPIC = 'clients_kafka'
BOOTSTRAP = 'kafka:9092'
ACCESS_KEY = 'minioadmin'
SECRET_KEY = 'minioadmin'

from clickhouse_driver import Client
file_path = 'KAFKA/topic=teste/year=2024/month=8/day=10/2024-08-10 21:45:28.json'
from minio import Minio
BUCKET_NAME = 'bronze-layer'
client = Minio("minio:9000",
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    secure = False
)

data = client.get_object(BUCKET_NAME, file_path)
content = json.loads(data.read().decode('utf-8'))
print(f"Conteúdo do arquivo {file_path}:")
print(content)



Conteúdo do arquivo KAFKA/topic=teste/year=2024/month=8/day=10/2024-08-10 21:45:28.json:
{"nome": "Ana J\u00falia", "sobrenome": "Barbosa", "idade": 31, "email": "cavalcantimaria-alice@example.net", "data_cadastro": "2024-02-20"}


In [22]:
content

'{"nome": "Ana J\\u00falia", "sobrenome": "Barbosa", "idade": 31, "email": "cavalcantimaria-alice@example.net", "data_cadastro": "2024-02-20"}'

In [34]:
from clickhouse_driver import Client

# Conexão ao servidor ClickHouse
client = Client(
    host='clickhouse',       # Endereço do servidor ClickHouse
    port=9000,              # Porta padrão para conexões TCP
    user='default',         # Usuário (padrão é 'default')
    password='',            # Senha (por padrão, vazio)
    database='default',     # Nome do banco de dados (por padrão, 'default')
)

In [101]:
client.execute("""CREATE OR REPLACE TABLE kafka_topics_offset_control (
                id UInt64,
                kafka_topic String,
                last_offset Int64,
                exec_date DateTime
                ) ENGINE = MergeTree()
                ORDER BY id """
        )
 

[]

In [93]:
client.execute("""INSERT INTO kafka_topics_offset_control (id, kafka_topic, last_offset, exec_date) VALUES 
(1, 'topic1', 12345, '2024-08-10'),
(2, 'topic2', 67890, '2024-08-10'),
(3, 'topic1', 12350, '2024-08-11'),
(4, 'topic3', 54321, '2024-08-11'),
(5, 'topic2', 67895, '2024-08-12');""")

[]

In [123]:
client.execute(f"SELECT * FROM  kafka_topics_offset_control")

[('teste', 1979, datetime.datetime(2024, 8, 10, 20, 59, 1)),
 ('teste', 1989, datetime.datetime(2024, 8, 10, 20, 59, 54))]

In [119]:


client.execute(f"SELECT max(last_offset) FROM kafka_topics_offset_control WHERE kafka_topic = 'TOPIC'")[0][0]


0